# Day 20
## Part 1

In [8]:
def parse_data(s):
    algorithm, image = s.strip().split('\n\n')
    
    algorithm = ''.join(algorithm.strip().splitlines())
    
    image_lit = set()
    for row, line in enumerate(image.splitlines()):
        for col, c in enumerate(line.strip()):
            if c == '#':
                image_lit.add((row, col))
                
    return algorithm, image_lit


def square(row, col):
    for dr in (-1, 0, 1):
        for dc in (-1, 0, 1):
            yield (row + dr, col + dc)
            
            
def step(algorithm, image):
    locations_to_check = set()
    for row, col in image:
        locations_to_check |= set(square(row, col))
    new_image = set()
    for row, col in locations_to_check:
        binary = ''.join('1' if (r, c) in image else '0'
                         for r, c in square(row, col))
        if algorithm[int(binary, 2)] == '#':
            new_image.add((row, col))
    return new_image


def print_image(image):
    min_row = min(row for row, _ in image)
    max_row = max(row for row, _ in image)
    min_col = min(col for _, col in image)
    max_col = max(col for _, col in image)
    s = ''
    for r in range(min_row, max_row + 1):
        for c in range(min_col, max_col + 1):
            s += '#' if (r, c) in image else '.'
        s += '\n'
    print(s)


def part_1(algorithm, image):
    for _ in range(2):
        image = step(algorithm, image)
    return len(image)


test_string = '''
..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###
'''
test_data = parse_data(test_string)
test_alg, test_img = test_data

print_image(test_img)

#..#.
#....
##..#
..#..
..###



In [4]:
new_image = step(test_alg, test_img)
print_image(new_image)

.##.##.
#..#.#.
##.#..#
####..#
.#..##.
..##..#
...#.#.



In [6]:
new_image = step(test_alg, new_image)
print_image(new_image)

.......#.
.#..#.#..
#.#...###
#...##.#.
#.....#.#
.#.#####.
..#.#####
...##.##.
....###..



In [7]:
assert part_1(*test_data) == 35

In [9]:
data = parse_data(open('input', 'r').read())
part_1(*data)

5243

It's turning on the lights when there's nothing around, i.e. for the real input `algorithm[0] == '#'`. A single step is meaningless as there are infinite lights. Let's try again.

In [14]:
def step(algorithm, image):
    min_row = min(row for row, _ in image)
    max_row = max(row for row, _ in image)
    min_col = min(col for _, col in image)
    max_col = max(col for _, col in image)

    new_image = set()
    for row in range(min_row - 1, max_row + 2):
        for col in range(min_col - 1, max_col + 2):
            binary = ''.join('1' if (r, c) in image else '0'
                             for r, c in square(row, col))
            if algorithm[int(binary, 2)] == '#':
                new_image.add((row, col))
    return new_image


def two_step(algorithm, image):
    # Step 1
    new_image = step(algorithm, image)
    min_row = min(row for row, _ in new_image)
    max_row = max(row for row, _ in new_image)
    min_col = min(col for _, col in new_image)
    max_col = max(col for _, col in new_image)
    
    # Draw a border of lights around the image
    for d in range(1, 4):
        new_image |= {(min_row - d, c) for c in range(min_col - 3, max_col + 4)}
        new_image |= {(max_row + d, c) for c in range(min_col - 3, max_col + 4)}
        new_image |= {(r, min_col - d) for r in range(min_row - 3, max_row + 4)}
        new_image |= {(r, max_col + d) for r in range(min_row - 3, max_row + 4)}
        
    new_image = step(algorithm, new_image)
    
    # Remove the border, allowing for an expansion of one in each direction
    return {(r, c) for r, c in new_image
            if min_row - 1 <= r <= max_row + 1
            and min_col - 1 <= c <= max_col + 1}


def part_1(algorithm, image):
    if algorithm[0] == '#':
        image = two_step(algorithm, image)
    else:
        for _ in range(2):
            image = step(algorithm, image)
    return len(image)


part_1(*test_data)

35

In [15]:
part_1(*data)

5486

## Part 2

In [16]:
def part_2(algorithm, image):
    if algorithm[0] == '#':
        for _ in range(25):
            image = two_step(algorithm, image)
    else:
        for _ in range(50):
            image = step(algorithm, image)
    return len(image)

assert part_2(*test_data) == 3351

In [17]:
part_2(*data)

20210

In [19]:
%%timeit 

part_2(*data)

6.96 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
